# Lokales Embedding benutzen

Dieses Notebook führt dich durch folgende Schritte:

- Installation der benötigten Pakete.
- Benutzen des Sitemap Loaders um die deepshore.de nach Wissensbeiträgen zu durchkämmen.
- Die Dokumente an ein *lokales* Embeddings Modell schicken und in Vectoren verwandeln und im Index speichern.
- Eine Frage an das *lokale* Embeddings Modell schicken und die neu gewonnenen Vectoren zusammen mit den indexierten Vectoren verorten und eine Antwort bekommen


Du benötigst:

- MacBook mit AppleSilicon (M1/M2)
- python 3.10
- Jupyter Notebook Server
- Einen OpenAI API Token

# Installation

In [ ]:
%pip install langchain=="0.0.279"
%pip install "langchain[llms]"=="0.0.279"
%pip install chromadb=="0.4.8"
%pip install llama_index=="0.8.17"
%pip install llama_hub=="0.0.26"
#!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python=="0.1.83" --upgrade --no-cache-dir --force-reinstall
%pip install mercury=="2.3.4"
%pip install sentence_transformers
%pip install gguf


# Modell herunterladen

In [ ]:
import urllib.request
import os
import os.path

model_name = f"{os.getcwd()}/models/openbuddy-llama2-13b-v11.1.Q6_K.gguf"

if not os.path.isfile(model_name):
    urllib.request.urlretrieve("https://huggingface.co/TheBloke/OpenBuddy-Llama2-13B-v11.1-GGUF/resolve/main/openbuddy-llama2-13b-v11.1.Q6_K.gguf", model_name)

## Dokumente laden

... indem man den Sitemap Loaders benutzt, um die Webseite deepshore.de nach Wissensbeiträgen zu durchkämmen

In [ ]:
from llama_hub.web.sitemap.base import SitemapReader

import nest_asyncio
nest_asyncio.apply()

loader = SitemapReader(html_to_text=True)
documents = loader.load_data(sitemap_url='https://deepshore.de/sitemap.xml', filter='https://deepshore.de/knowledge/2023-05-08')

print(len(documents))

# Model laden

Folgendes modell wurde hier heruntergeladen: [hugging face](https://huggingface.co/TheBloke/open-llama-7b-open-instruct-GGML)

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager


callbacks = [StreamingStdOutCallbackHandler()]
callback_manager = CallbackManager(callbacks)

llama_llm = LlamaCpp(model_path=model_name, callback_manager=callback_manager, use_mlock=False, n_threads=2, n_gpu_layers=32, temperature=0.25, verbose=True, n_ctx=2048)

prompt = """
Frage: Wieviele Bundesländer hat Deutschland?
"""
llama_llm(prompt)

## Die Dokumente an das lokale Embeddings Modell schicken

...und in Vektoren verwandeln und im Index speichern 

In [ ]:
from langchain.vectorstores import Chroma
from llama_index.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma

import os
from langchain.embeddings.llamacpp import LlamaCppEmbeddings
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter, SpacyTextSplitter,TokenTextSplitter
from langchain.vectorstores import Chroma
from llama_index.schema import Document
from llama_index.indices.service_context import ServiceContext

langchain_documents = []
for d in documents:
    langchain_documents.append(d.to_langchain_format())

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(langchain_documents)

llama_documents = []
for langchain_doc in docs:
    llama_documents.append(Document.from_langchain_format(langchain_doc))

print(len(llama_documents))

# create the open-source embedding function
embedding = LlamaCppEmbeddings(model_path=model_name, n_ctx=2048, use_mlock=False, n_gpu_layers=32, n_threads=8, f16_kv=True)

#https://docs.trychroma.com/telemetry#opting-out
chromadb_settings = Settings(anonymized_telemetry=False, persist_directory=f"{os.getcwd()}/chroma", is_persistent=True)
chromadb_client = chromadb.Client(chromadb_settings)
chroma = Chroma(collection_name='deepshore-sitemap', client=chromadb_client, embedding_function=embedding)

chroma_collection= chromadb_client.get_collection(name='deepshore-sitemap')




In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings.llamacpp import LlamaCppEmbeddings
from llama_index.llm_predictor.base import LLMPredictor
from llama_index.vector_stores import ChromaVectorStore
from llama_index.vector_stores.types import VectorStoreQuery, VectorStoreQueryResult
from llama_index.indices.prompt_helper import PromptHelper
from llama_index import set_global_service_context

llmpredictor = LLMPredictor(llm = llama_llm, callback_manager=callback_manager)

chromastore = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=chromastore)
service_context = ServiceContext.from_defaults(llm_predictor=llmpredictor, embed_model=embedding)
set_global_service_context(service_context=service_context)

index = VectorStoreIndex.from_vector_store(vector_store=chromastore)

index = VectorStoreIndex.from_documents(
    documents=documents, storage_context=storage_context, service_context=service_context
)


## Eine Frage an das Modell schicken

... und eine Antwort im Kontext der Daten bekommen

In [ ]:
import textwrap
import mercury as mr
 
# set Application parameters
app = mr.App(title="",
        description="",
        show_code=False,
        show_prompt=False,
        continuous_update=True,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

question = mr.Text(value="Was ist k6.io? Wofür benutzt man es?", label="Womit kann Deepshore helfen?", rows=1)

query_engine = index.as_query_engine(streaming=False)

response = query_engine.query(question.value)

messages = [question.value, "\n".join(textwrap.wrap(response.response,29))]
mr.Chat(messages)